## This notebook can be used to extract entries of .pdb files, for example, resolution of selected pdb files.

In [1]:
#import biotite.database.rcsb as rcsb
#Ligand Expo website (http://ligand-expo.rcsb.org/) will be offline in January 2023
import redo
from tqdm.auto import tqdm
import pypdb
import pandas as pd
import numpy as np

In [2]:
# pip install biotite
# pip install redo
# pip install pypdb

In [3]:
df = pd.read_excel('dataset.xlsx')
df.head(10)

,Unnamed: 0,Compound Name,index,VFVS_INPUT,Unnamed: 3,smiles,IC50 (μM),IC50 (μM) ROUND,Kd (μM) (exp),Ki (μM),Crystal structure (Y/N),pdb,covalent (Y/N),resolution,URL
0,0,X77 (rac),M001,AAA001,20.0,CC(C)(C)c1ccc(cc1)N([C@H](c2cccnc2)C(=O)NC3CCC...,2.8 ± 0.6,2.8,1.9 ± 0.1,N/D,Y,6W63,N,2.1,https://doi.org/10.1021/jacs.1c08402
1,1,ML188 (rac),M002,AAA002,20.0,CC(C)(C)c1ccc(cc1)N([C@H](c2cccnc2)C(=O)NC(C)(...,6.7 ± 1.3,6.7,5.7 ± 0.8,N/D,Y,7L0D,N,2.39,NaN
2,2,NaN,M003,AAA003,20.0,c1c(cncc1F)N2C(=O)[C@@H](NC2=O)CC3CCCCC3,26 ± 5%,N/D,22.9 ± 1.8,N/D,Y,7B2U,N,1.55,NaN
3,3,NaN,M004,AAA004,20.0,CCC1CN(CCN1C(=O)C1=C2C=NNC2=CC=C1)C1=CC=CC=C1,55 ± 2%,N/D,31.1 ± 1.8,N/D,N,N/D,N,N/D,NaN
4,4,NaN,M005,AAA005,20.0,c1cc2c(cc1Cl)CC[C@@]23C(=O)N(C(=O)N3)Cc4ccon4,63 ± 3%,N/D,60.7 ± 5.3,N/D,Y,7AU4,N,1.82,NaN
5,5,NaN,M006,AAA006,20.0,CN1CCN(CC1)C(=O)Cc2c[nH]c3c2cccn3,101 ± 1%,N/D,>200,N/D,Y,5RF7,N,1.54,NaN
6,6,NaN,M007,AAA007,20.0,CC1CCN(CC1)C(=O)Cn2c3ccccc3nn2,20 ± 14,20,7.2 ± 1,N/D,Y,7B2J,N,1.55,NaN
7,7,NaN,M008,AAA008,20.0,C=C1CCN(CC1)C(=O)Cn2c3ccccc3nn2,44 ± 4%,N/D,38.5 ± 3.6,N/D,Y,7B5Z,N,1.65,NaN
8,8,NaN,M009,AAA009,20.0,O=C(CN1N=NC2=C1C=CC=C2)N1CCCC1C1=CC=CC=C1,54 ± 2%,N/D,71.7 ± 6.8,N/D,N,N/D,N,N/D,NaN
9,9,NaN,M010,AAA010,20.0,CCN(Cc1ccoc1)C(=O)Cn2c3ccccc3nn2,61 ± 3%,N/D,79.3 ± 6.4,N/D,Y,7B77,N,1.6,NaN


In [5]:
pdb_ids = df['pdb']
pdb_ids = pdb_ids.to_list()

In [6]:
#query = rcsb.FieldQuery(
#    "rcsb_entry_container_identifiers.entry_id",
#    exact_match=pdb_id,
#)

#pdb_ids = rcsb.search(query)
#print(rcsb.search(query))

In [7]:
@redo.retriable(attempts=10, sleeptime=2)
def describe_one_pdb_id(pdb_id):
    """Fetch meta information from PDB."""
    described = pypdb.describe_pdb(pdb_id)
    if described is None:
        print(f"! Error while fetching {pdb_id}, retrying ...")
        raise ValueError(f"Could not fetch PDB id {pdb_id}")
    return described

In [8]:
i = 0
for pdb_id in pdb_ids:
    if pdb_id == 'N/D' or pdb_id == '8B56':
        df.at[i, 'resolution'] = 'N/D'
    else:
        pdb_data = describe_one_pdb_id(pdb_id)
        df.at[i, 'resolution'] = pdb_data["rcsb_entry_info"]["resolution_combined"][0]
        #print(pdb_data["rcsb_entry_info"]["resolution_combined"][0])
        #print(pdb_id)
    
    i += 1

In [9]:
#This will show all the keys of the dictionary.

#pdb_id = '7VU6'
#pdb_data = describe_one_pdb_id(pdb_id)
#print("\n".join(pdb_data.keys()))

In [ ]:
#This can be used to create a new dataframe.

#pdbs_data = [describe_one_pdb_id(pdb_id) for pdb_id in tqdm(pdb_ids)]

#resolution = pd.DataFrame(
#    [
#        [pdb_data["entry"]["id"], pdb_data["rcsb_entry_info"]["resolution_combined"][0]]
#        for pdb_data in pdbs_data
#    ],
#    columns=["pdb_id", "resolution"],
#).sort_values(by="resolution", ignore_index=True)
#resolution

In [10]:
df.head(20)

,Unnamed: 0,Compound Name,index,VFVS_INPUT,Unnamed: 3,smiles,IC50 (μM),IC50 (μM) ROUND,Kd (μM) (exp),Ki (μM),Crystal structure (Y/N),pdb,covalent (Y/N),resolution,URL
0,0,X77 (rac),M001,AAA001,20.0,CC(C)(C)c1ccc(cc1)N([C@H](c2cccnc2)C(=O)NC3CCC...,2.8 ± 0.6,2.8,1.9 ± 0.1,N/D,Y,6W63,N,2.1,https://doi.org/10.1021/jacs.1c08402
1,1,ML188 (rac),M002,AAA002,20.0,CC(C)(C)c1ccc(cc1)N([C@H](c2cccnc2)C(=O)NC(C)(...,6.7 ± 1.3,6.7,5.7 ± 0.8,N/D,Y,7L0D,N,2.39,NaN
2,2,NaN,M003,AAA003,20.0,c1c(cncc1F)N2C(=O)[C@@H](NC2=O)CC3CCCCC3,26 ± 5%,N/D,22.9 ± 1.8,N/D,Y,7B2U,N,1.55,NaN
3,3,NaN,M004,AAA004,20.0,CCC1CN(CCN1C(=O)C1=C2C=NNC2=CC=C1)C1=CC=CC=C1,55 ± 2%,N/D,31.1 ± 1.8,N/D,N,N/D,N,N/D,NaN
4,4,NaN,M005,AAA005,20.0,c1cc2c(cc1Cl)CC[C@@]23C(=O)N(C(=O)N3)Cc4ccon4,63 ± 3%,N/D,60.7 ± 5.3,N/D,Y,7AU4,N,1.82,NaN
5,5,NaN,M006,AAA006,20.0,CN1CCN(CC1)C(=O)Cc2c[nH]c3c2cccn3,101 ± 1%,N/D,>200,N/D,Y,5RF7,N,1.54,NaN
6,6,NaN,M007,AAA007,20.0,CC1CCN(CC1)C(=O)Cn2c3ccccc3nn2,20 ± 14,20,7.2 ± 1,N/D,Y,7B2J,N,1.55,NaN
7,7,NaN,M008,AAA008,20.0,C=C1CCN(CC1)C(=O)Cn2c3ccccc3nn2,44 ± 4%,N/D,38.5 ± 3.6,N/D,Y,7B5Z,N,1.65,NaN
8,8,NaN,M009,AAA009,20.0,O=C(CN1N=NC2=C1C=CC=C2)N1CCCC1C1=CC=CC=C1,54 ± 2%,N/D,71.7 ± 6.8,N/D,N,N/D,N,N/D,NaN
9,9,NaN,M010,AAA010,20.0,CCN(Cc1ccoc1)C(=O)Cn2c3ccccc3nn2,61 ± 3%,N/D,79.3 ± 6.4,N/D,Y,7B77,N,1.6,NaN


In [11]:
df.to_excel('dataset.xlsx')